In [ ]:
# df.loc[idx[slice(None), '2012-02-15'],]['cs_pctrank_ts_argmax5_closeadj'].unique()  -> for indexing multiindex

In [1]:
import pandas as pd
import nasdaqdatalink as ndl
import json
import numpy as np
from typing import List, Callable, Dict
from realgam.quantlib import general_utils as gu
from realgam.quantlib.engineer.op_engineer_vect import OpEngineerV
from realgam.quantlib.engineer.ta_engineer_vect import TalibEngineerV
from realgam.quantlib.engineer.alpha_engineer_vect import AlphaEngineerV
# from realgam.quantlib.engineer.test_no_class import alpha001, ts_ret
import talib
from joblib import Parallel, delayed
import datetime
import os
import time
PROJECT_PATH = os.getenv('QuantSystemMVP')
DATA_PATH = f'{PROJECT_PATH}/Data/stock_hist.obj'
NDL_CONFIG_PATH = f'{PROJECT_PATH}/realgam/quantlib/nasdaq_dl_config.json'
NDL_CONFIG = json.load(open(NDL_CONFIG_PATH))

In [2]:
stocks_df, stocks_extended_df, available_tickers = gu.load_file(DATA_PATH)
stacked_hist = stocks_df.copy()

In [35]:
stocks_df, stocks_extended_df, available_tickers = gu.load_file(f'{PROJECT_PATH}/Data/stock_hist_perma.obj')

In [36]:
small_hist = stocks_df.reset_index()

In [37]:
small_hist.date.max()

Timestamp('2022-03-25 00:00:00')

In [38]:
small_hist[small_hist.date== '2022-03-25']

,permaticker,date,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume
3323,104612,2022-03-25,APCX,1.27,1.270,1.20,1.22,1.27,1.270,1.20,1.22,45342.0
8218,105149,2022-03-25,FNGR,2.50,2.750,2.40,2.56,2.50,2.750,2.40,2.56,10103.0
13369,107145,2022-03-25,VIVK,2.51,2.800,2.28,2.49,2.51,2.800,2.28,2.49,201201.0
13711,107345,2022-03-25,INVO,2.56,2.652,2.56,2.59,2.56,2.652,2.56,2.59,9985.0
13965,107353,2022-03-25,WMPN,12.80,12.870,12.80,12.83,12.80,12.870,12.80,12.83,24828.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12003519,638344,2022-03-25,ATAKU,10.05,10.050,10.05,10.05,10.05,10.050,10.05,10.05,113.0
12003524,638348,2022-03-25,GHIXU,10.05,10.050,10.05,10.05,10.05,10.050,10.05,10.05,1551.0
12003526,638355,2022-03-25,RFACU,10.03,10.040,10.03,10.03,10.03,10.040,10.03,10.03,375467.0
12003528,638356,2022-03-25,ACAXU,10.05,10.090,10.05,10.09,10.05,10.090,10.05,10.09,164263.0


In [42]:
small_hist[small_hist.ticker== 'AAPL'].date.max()


Timestamp('2022-03-25 00:00:00')

In [45]:
small_hist[small_hist.duplicated(['permaticker', 'date'])]

,permaticker,date,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume


In [39]:
old_stocks_df[old_stocks_df.ticker == 'HSGH']

NameError: name 'old_stocks_df' is not defined

In [132]:
old_stocks_df[old_stocks_df.duplicated(['permaticker', 'date'])]

,permaticker,date,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume


In [3]:
stacked_hist = stacked_hist.reset_index().set_index(['ticker', 'date'])

In [77]:
stacked_hist = stacked_hist.reset_index()

In [43]:
stacked_hist

NameError: name 'stacked_hist' is not defined

In [4]:
stacked_hist[stacked_hist.index.isin(['GGE'], level='ticker')]

open  high   low  close  openadj  highadj  lowadj  \
ticker date                                                            
GGE    2022-03-21  3.00  3.05  2.95  2.965     3.00     3.05    2.95   
       2022-03-22  2.90  3.20  2.90  3.020     2.90     3.20    2.90   
       2022-03-23  3.23  3.23  2.86  3.000     3.23     3.23    2.86   
       2022-03-24  2.99  3.32  2.70  3.300     2.99     3.32    2.70   

                   closeadj    volume  
ticker date                            
GGE    2022-03-21     2.965   25566.0  
       2022-03-22     3.020   46899.0  
       2022-03-23     3.000   19662.0  
       2022-03-24     3.300  535484.0

In [5]:
tickers_metadata = ndl.get_table('SHARADAR/TICKERS', paginate=True)

In [139]:
tickers_metadata[tickers_metadata.ticker == 'HGSH']

,table,permaticker,ticker,name,exchange,isdelisted,category,cusips,siccode,sicsector,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
None,,,,,,,,,,,,,,,,,,,,,


In [122]:
focused_cat = tickers_metadata[tickers_metadata.category.isin(["Domestic Common Stock",
                  "Domestic Common Stock Primary Class", "Canadian Common Stock",
                  "Canadian Common Stock Primary Class",
                  "Domestic Common Stock Secondary Class"])]

In [141]:
focused_cat[focused_cat.ticker == 'GGE']

,table,permaticker,ticker,name,exchange,isdelisted,category,cusips,siccode,sicsector,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
None,,,,,,,,,,,,,,,,,,,,,
6117,SF1,195736,GGE,Green Giant Inc,NASDAQ,N,Domestic Common Stock,16948W100 16948W209 16937D105,6500.0,Finance Insurance And Real Estate,...,USD,China,2022-03-23,2015-12-11,2008-04-01,2022-03-24,2006-09-30,2021-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.chinahgs.com
24116,SEP,195736,GGE,Green Giant Inc,NASDAQ,N,Domestic Common Stock,16948W100 16948W209 16937D105,6500.0,Finance Insurance And Real Estate,...,USD,China,2022-03-24,2015-12-11,2008-04-01,2022-03-24,2006-09-30,2021-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.chinahgs.com


In [121]:
tickers_metadata[tickers_metadata.category == 'Canadian Common Stock Primary Class']

,table,permaticker,ticker,name,exchange,isdelisted,category,cusips,siccode,sicsector,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
None,,,,,,,,,,,,,,,,,,,,,
628,SF1,106711,ALLSA,Allstream Inc,NASDAQ,Y,Canadian Common Stock Primary Class,02004C105,4813.0,Transportation Communications Electric Gas And...,...,CAD,Alberta; Canada,2021-06-14,2021-06-12,2003-04-01,2004-06-04,2001-12-31,2004-03-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,None
1009,SF1,123729,AQN,Algonquin Power & Utilities Corp,NYSE,N,Canadian Common Stock Primary Class,015857105,4911.0,Transportation Communications Electric Gas And...,...,USD,Ontario; Canada,2022-03-07,2017-07-18,2009-12-29,2022-03-24,2008-12-31,2021-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://algonquinpower.com
1545,SF1,167282,BAM,Brookfield Asset Management Inc,NYSE,N,Canadian Common Stock Primary Class,112585104 10549P606 280905803,6512.0,Finance Insurance And Real Estate,...,USD,Ontario; Canada,2022-02-10,2016-01-17,1986-01-01,2022-03-24,2000-12-31,2021-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.brookfield.com
1657,SF1,633918,BCTX,BriaCell Therapeutics Corp,NASDAQ,N,Canadian Common Stock Primary Class,10778Y302,2834.0,Manufacturing,...,CAD,British Columbia; Canada,2022-03-15,2021-02-24,2021-02-24,2022-03-24,2019-06-30,2021-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,None
3829,SF1,165999,CVE,Cenovus Energy Inc,NYSE,N,Canadian Common Stock Primary Class,15135U109,1311.0,Mining,...,CAD,Alberta; Canada,2022-03-24,2015-12-31,2009-11-17,2022-03-24,2008-09-30,2021-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.cenovus.com
6102,SF1,110038,GFL,GFL Environmental Inc,NYSE,N,Canadian Common Stock Primary Class,36168Q104,4953.0,Transportation Communications Electric Gas And...,...,CAD,Ontario; Canada,2022-02-10,2020-03-11,2020-03-03,2022-03-24,2018-12-31,2021-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.gflenv.com
6111,SF1,124620,GG,Goldcorp Inc,NYSE,Y,Canadian Common Stock Primary Class,380956409 380956102,1040.0,Mining,...,USD,British Columbia; Canada,2019-03-28,2016-11-17,1995-08-18,2019-04-17,2000-12-31,2018-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,None
11451,SF1,119663,PLM,Polymet Mining Corp,NYSEMKT,N,Canadian Common Stock Primary Class,731916102 731916409,1000.0,Mining,...,USD,Ontario; Canada,2022-03-17,2018-04-21,2000-10-12,2022-03-24,2004-09-30,2021-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.polymetmining.com
12667,SF1,197204,RY,Royal Bank Of Canada,NYSE,N,Canadian Common Stock Primary Class,780087102,6029.0,Finance Insurance And Real Estate,...,CAD,Ontario; Canada,2022-03-07,2016-08-06,1995-10-16,2022-03-24,2001-06-30,2021-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.rbc.com


In [49]:
tickers_metadata['permaticker'] = tickers_metadata.permaticker.astype('str')

In [69]:
# Filtering exchanges and categories of stocks
filter_exchange = NDL_CONFIG['exchanges']

filter_cat = NDL_CONFIG['stock_categories']
# Filter stocks that we focus on
focused_stocks_ticker = tickers_metadata[
    (tickers_metadata.exchange.isin(filter_exchange)) & (
        tickers_metadata.category.isin(filter_cat))][['ticker', 'permaticker', 'relatedtickers']].drop_duplicates()


In [99]:
tickers_metadata[tickers_metadata.ticker == 'ALTM']

,table,permaticker,ticker,name,exchange,isdelisted,category,cusips,siccode,sicsector,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
None,,,,,,,,,,,,,,,,,,,,,


In [70]:
ticker_list = []
permaticker_list = []
none_list = []
for index, row in focused_stocks_ticker.iterrows():
    # print(type(row['relatedtickers']))
    if row['relatedtickers']:
        for ticker in row['relatedtickers'].split(' '):
            ticker_list.append(ticker)
            permaticker_list.append(row['permaticker'])
            none_list.append(None)

In [82]:
temp_df = pd.DataFrame()
temp_df['ticker'] = ticker_list
temp_df['permaticker'] = permaticker_list
temp_df['relatedtickers'] = none_list
temp_df = temp_df[~temp_df.isin(focused_stocks_ticker.ticker)]
temp_df.dropna(inplace=True)

In [91]:
permaticker_ticker = pd.concat([focused_stocks_ticker, temp_df])


In [90]:
temp_df[temp_df.ticker == 'A']

,ticker,permaticker,relatedtickers


In [84]:
temp_df[temp_df.isin(list(stacked_hist.ticker.unique()))]

,ticker,permaticker,relatedtickers


In [74]:
temp_df[temp_df.duplicated('ticker')]

,ticker,permaticker,relatedtickers
27,AAII,116429,None
59,ABI,115555,None
83,ABRX,115544,None
88,ABH,180839,None
107,ABD,198896,None
...,...,...,...
15007,ZING,637216,None
15008,ZINGW,637216,None
15009,ZNTEW,634235,None
15011,ZTAQW,636032,None


In [112]:
stacked_hist[~stacked_hist.ticker.isin(permaticker_ticker.ticker.unique())].ticker.unique()

array(['AFIN', 'ALTM', 'BFRA', 'BOMN', 'BTNB', 'CINR', 'FIG', 'GCAC',
       'GRUB', 'HCCC', 'HFC', 'HGSH', 'IGAP', 'MOTV', 'NXTD', 'NXU',
       'PLIN', 'REPH', 'ROCR', 'SBEA', 'SNII', 'SPAQ', 'TVAC', 'VIAC',
       'VIACA'], dtype=object)

In [123]:
stacked_hist.date.max()

Timestamp('2022-03-24 00:00:00')

In [60]:
permaticker_ticker_nodup = permaticker_ticker.drop_duplicates('ticker', keep='first')

In [125]:
stacked_hist = stacked_hist.set_index('date')

In [126]:
merged = pd.merge(stacked_hist, permaticker_ticker[['ticker', 'permaticker']], how='left', on=['ticker'])

In [127]:
merged

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,permaticker
0,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0,196290
1,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0,196290
2,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0,196290
3,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0,196290
4,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0,196290
...,...,...,...,...,...,...,...,...,...,...,...
13860865,ZZ,2.20,2.210,2.200,2.20,2.200000,2.210000,2.200000,2.200,407100.0,198673
13860866,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,160000.0,198673
13860867,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,308900.0,198673
13860868,ZZ,2.20,2.210,2.200,2.21,2.200000,2.210000,2.200000,2.210,275000.0,198673


In [95]:
merged[merged.duplicated(['permaticker', 'date'])].ticker.unique()

array(['ALTM', 'BFRA', 'BOMN', 'BTNB', 'CINR', 'FIG', 'GCAC', 'GRUB',
       'HCCC', 'HFC', 'HGSH', 'IGAP', 'MOTV', 'NXTD', 'NXU', 'PLIN',
       'REPH', 'ROCR', 'SBEA', 'SNII', 'SPAQ', 'TVAC', 'VIAC', 'VIACA'],
      dtype=object)

In [109]:
tickers_metadata[tickers_metadata.ticker == 'SPAQ']

,table,permaticker,ticker,name,exchange,isdelisted,category,cusips,siccode,sicsector,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
None,,,,,,,,,,,,,,,,,,,,,


In [107]:
data = ndl.get_table('SHARADAR/SEP', ticker='A',
                                     paginate=True, date={'gte': '2012-01-01'})

In [108]:
data

,ticker,date,open,high,low,close,volume,closeadj,closeunadj,lastupdated
None,,,,,,,,,,
0,A,2022-03-24,134.89,136.580,133.590,136.41,1346742.0,136.410,136.41,2022-03-24
1,A,2022-03-23,137.85,137.955,133.448,134.18,1336815.0,134.180,134.18,2022-03-23
2,A,2022-03-22,138.01,140.200,137.650,139.29,1977714.0,139.290,139.29,2022-03-22
3,A,2022-03-21,137.63,139.790,137.010,138.14,1285102.0,138.140,138.14,2022-03-21
4,A,2022-03-18,136.21,139.393,136.210,139.12,2761938.0,139.120,139.12,2022-03-18
...,...,...,...,...,...,...,...,...,...,...
2569,A,2012-01-09,37.62,38.420,37.472,38.38,3168500.0,25.328,38.38,2022-01-03
2570,A,2012-01-06,37.09,37.555,36.805,37.40,3370100.0,24.681,37.40,2022-01-03
2571,A,2012-01-05,35.88,37.320,35.720,37.00,4894600.0,24.417,37.00,2022-01-03


In [38]:
merged.sort_values(['permaticker', 'date'], inplace=True)

In [39]:
merged[merged.permaticker == '195736']

,ticker,date,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,permaticker
7229010,HGSH,2012-01-03,1.360,1.360,1.240,1.266,1.360,1.360,1.240,1.266,1050.0,195736
7229011,HGSH,2012-01-04,1.223,1.356,1.223,1.261,1.223,1.356,1.223,1.261,450.0,195736
7229012,HGSH,2012-01-05,1.360,1.360,1.360,1.360,1.360,1.360,1.360,1.360,150.0,195736
7229013,HGSH,2012-01-06,1.360,1.360,1.260,1.360,1.360,1.360,1.260,1.360,6300.0,195736
7229014,HGSH,2012-01-09,1.314,1.314,1.300,1.300,1.314,1.314,1.300,1.300,250.0,195736
...,...,...,...,...,...,...,...,...,...,...,...,...
7231579,HGSH,2022-03-18,3.350,3.590,2.870,3.090,3.350,3.590,2.870,3.090,148358.0,195736
6503694,GGE,2022-03-21,3.000,3.050,2.950,2.965,3.000,3.050,2.950,2.965,25566.0,195736
6503695,GGE,2022-03-22,2.900,3.200,2.900,3.020,2.900,3.200,2.900,3.020,46899.0,195736
6503696,GGE,2022-03-23,3.230,3.230,2.860,3.000,3.230,3.230,2.860,3.000,19662.0,195736


In [8]:
tickers_action = ndl.get_table('SHARADAR/ACTIONS', paginate=True)

In [9]:
tickers_change = tickers_action[tickers_action.action.isin(['tickerchangefrom'])]

In [41]:
tickers_change[tickers_change.ticker == 'HGSH']

,date,action,ticker,name,value,contraticker,contraname
None,,,,,,,


In [5]:
ae = AlphaEngineerV(small_hist)

In [9]:
ae.alpha7()

ticker  date      
A       2012-04-09    23.0
AAIIQ   2012-04-09    -0.0
AAMRQ   2012-04-09    -0.0
AAON    2012-04-09    51.0
AATC    2012-04-09   -28.0
                      ... 
ABSI    2022-03-21    59.0
ABST    2022-03-21     7.0
ACB     2022-03-21    -9.0
ACEL    2022-03-21    40.0
ACER    2022-03-21   -51.0
Length: 41144, dtype: float64